In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
df = pd.read_csv('season1.csv',index_col=0)

def clean_separate_df(df):
    all_players = df[df.year==2019][['player','pos','adp','ppr_pts','year']].sort_values('adp').reset_index(drop=True)
    qb = all_players[all_players.pos=='QB']
    rb = all_players[all_players.pos=='RB']
    wr = all_players[all_players.pos=='WR']
    te = all_players[all_players.pos=='TE']
    return all_players, qb, rb, wr, te





In [3]:
def create_strats(strat):
    if strat == 'hero':
        return ['hero_rb', ['RB','TE','QB','WR','WR','WR','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL'].copy()]
    if strat == 'zero':
        return ['zero_rb', ['WR','WR','TE','QB','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL'].copy()]
    if strat == 'rb_heavy':
        return ['rb_heavy', ['RB','RB','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL'].copy()]
    else:
        return ['value',['VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL','VAL'].copy()]
    
def create_owners_strats(hero,zero,rb_heavy,value):
    strats = []
    for i in range(0,hero):
        strats.append(create_strats('hero'))
    for i in range(0,zero):
        strats.append(create_strats('zero'))
    for i in range(0,rb_heavy):
        strats.append(create_strats('rb_heavy'))
    for i in range(0,value):
        strats.append(create_strats('value'))
    return strats

In [4]:
strats = create_owners_strats(4,4,0,4)

In [5]:
strats[0]

['hero_rb',
 ['RB',
  'TE',
  'QB',
  'WR',
  'WR',
  'WR',
  'VAL',
  'VAL',
  'VAL',
  'VAL',
  'VAL',
  'VAL',
  'VAL',
  'VAL']]

In [6]:
class Cheatsheet:
    def __init__(self, all_players, qb, rb, wr, te):
        self.all_players = all_players
        self.qb = qb
        self.rb = rb
        self.wr = wr
        self.te = te
      
    def draft_qb(self,owner):
        selection = self.qb.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.qb = self.qb[1:]
        owner.qb_count+=1
        return selection
        
    def draft_rb(self,owner):
        selection = self.rb.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.rb = self.rb[1:]
        owner.rb_count+=1
        return selection
    
    def draft_wr(self,owner):
        selection = self.wr.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.wr = self.wr[1:]
        owner.wr_count+=1
        return selection
    
    def draft_te(self,owner):
        selection = self.te.iloc[0,:]
        self.all_players = self.all_players.drop(index = selection.name)
        self.te = self.te[1:]
        owner.te_count+=1
        return selection
    
    def draft_not_qb(self,owner):
        i = 0
        while self.all_players.iloc[i,1] == 'QB':
            i+=1
        if self.all_players.iloc[i,1] == 'RB':
            return self.draft_rb(owner)
        elif self.all_players.iloc[i,1] == 'WR':
            return self.draft_wr(owner)
        else:
            return self.draft_te(owner)
    
    def draft_not_te(self,owner):
        i = 0
        while self.all_players.iloc[i,1] == 'TE':
            i+=1
        if self.all_players.iloc[i,1] == 'RB':
            return self.draft_rb(owner)
        elif self.all_players.iloc[i,1] == 'WR':
            return self.draft_wr(owner)
        else:
            return self.draft_qb(owner)
        
    def draft_not_wr(self,owner):
        pass
    
    def draft_value(self,owner): 
        selection = self.all_players.iloc[0,:]
        if self.all_players.iloc[0,:].pos == 'QB':
            self.qb = self.qb.drop(index = selection.name)
            owner.qb_count+=1
        elif self.all_players.iloc[0,:].pos == 'RB':
            self.rb = self.rb.drop(index = selection.name)
            owner.rb_count+=1
        elif self.all_players.iloc[0,:].pos == 'WR':
            self.wr = self.wr.drop(index = selection.name)
            owner.wr_count+=1
        else:
            self.te = self.te.drop(index = selection.name)
            owner.te_count+=1
        self.all_players = self.all_players.iloc[1:]
        return selection
    

class Team_owner:
    def __init__(self, strat, draft_pos, draft_num):
        self.team = []
        self.strat_name = strat[0]      
        self.strat_order = strat[1]
        self.draft_pos = draft_pos
        self.draft_num = draft_num
        self.qb_count = 0
        self.te_count = 0
        self.wr_count = 0
        self.rb_count = 0
        
        
    def draft_player(self,cheatsheet):
        if len(self.team) == 15 and self.te_count == 0: #Draft a TE in 10th round if no TEs
            self.strat_order.pop(0)
            return cheatsheet.draft_te(self)
        
        elif len(self.team) == 8 and self.qb_count == 0: #Draft a QB in 9th round if no QBs
            self.strat_order.pop(0)
            return cheatsheet.draft_qb(self)
        
        elif len(self.team) == 7 and self.rb_count == 1: #Draft a RB in 8th round if only 1 RB
            self.strat_order.pop(0)
            return cheatsheet.draft_rb(self)
        
        elif len(self.team) == 7 and self.wr_count == 1: #Draft a WR in 8th round if only 1 WR
            self.strat_order.pop(0)
            return cheatsheet.draft_wr(self)
        
        elif len(self.team) == 6 and self.rb_count == 0: #Draft a RB in 7th round if only 0 RB
            self.strat_order.pop(0)
            return cheatsheet.draft_rb(self)
        
        elif len(self.team) == 7 and self.wr_count == 0: #Draft a WR in 7th round if only 1 WR
            self.strat_order.pop(0)
            return cheatsheet.draft_wr(self)
        
        elif self.qb_count == 2 and self.strat_order[0]=='VAL': #Don't draft more than 2 QB
            self.strat_order.pop(0)
            return cheatsheet.draft_not_qb(self)
        
        elif self.te_count == 2 and self.strat_order[0]=='VAL': # Don't draft more than 2 TE
            self.strat_order.pop(0)
            return cheatsheet.draft_not_te(self)
        
        
#         elif self.rb_count > 3:
#             self.strat_order.pop(0)
#             return cheatsheet.draft_wr(self)
        
#         elif self.wr_count > 3:
#             self.strat_order.pop(0)
#             return cheatsheet.draft_rb(self)
        
        else:
            pos = self.strat_order.pop(0)
            if pos == 'QB':
                self.qb_count+=1
                return cheatsheet.draft_qb(self)
            elif pos == 'RB':
                return cheatsheet.draft_rb(self)
            elif pos == 'WR':
                return cheatsheet.draft_wr(self)
            elif pos == 'TE':
                return cheatsheet.draft_te(self)
            else:
                return cheatsheet.draft_value(self)


def top7(df):
    '''
    Takes in a team owner's team and returns their best starters for the season:
    1 QB, 2 RB, 2 WR, 1 RB or WR, 1 TE for a total of 7 players 
    '''
    top7 = [] 
    for i in range(1,13):
        top7.append(df[(df.draft_pos==i) & (df.pos=='QB')].sort_values('ppr_pts',ascending=False).head(1))
        top7.append(df[(df.draft_pos==i) & (df.pos=='RB')].sort_values('ppr_pts',ascending=False).head(2))
        df = df.drop(df[(df.draft_pos==i) & (df.pos=='RB')].sort_values('ppr_pts',ascending=False).head(2).index)
        top7.append(df[(df.draft_pos==i) & (df.pos=='WR')].sort_values('ppr_pts',ascending=False).head(2))
        df = df.drop(df[(df.draft_pos==i) & (df.pos=='WR')].sort_values('ppr_pts',ascending=False).head(2).index)
        top7.append(df[(df.draft_pos==i) & ((df.pos=='RB')|(df.pos=='WR'))].sort_values('ppr_pts',ascending=False).head(1))
        top7.append(df[(df.draft_pos==i) & (df.pos=='TE')].sort_values('ppr_pts',ascending=False).head(1))
    return pd.concat(top7)
            
    

In [7]:
all_players, qb, rb, wr, te = clean_separate_df(df)

# strats = create_owners_strats(2,2,2,6)
# np.random.shuffle(strats)


In [8]:
len(strats[0][1])

14

In [9]:
cheat = Cheatsheet(all_players, qb, rb, wr, te) # Create new cheat sheet

owners = [] #Create 12 team owners with 'value_strat' drafting strategy
for i, s in zip(range(0,12),strats.copy()):
    owners.append(Team_owner(s,i+1,1))

In [10]:
owners.reverse()

In [11]:

for i in range(0,14): #Draft 8 rounds for each team owner
    owners.reverse()
    for o in owners:
        o.team.append(o.draft_player(cheat))

In [12]:
all_teams = []

for o in owners:
    df = pd.concat(o.team,axis=1).T
    df['draft_pos'] = o.draft_pos
    df['draft_strat'] = o.strat_name
    df['draft_num'] = o.draft_num
    all_teams.append(df)

df = pd.concat(all_teams)

In [13]:
df[df.draft_pos==4].sort_values('adp')

,player,pos,adp,ppr_pts,year,draft_pos,draft_strat,draft_num
3,Ezekiel Elliott,RB,4.0,311.7,2019,4,hero_rb,1
25,George Kittle,TE,28.0,222.5,2019,4,hero_rb,1
32,Amari Cooper,WR,35.0,246.5,2019,4,hero_rb,1
45,Kenny Golladay,WR,48.0,248.0,2019,4,hero_rb,1
59,Mike Williams,WR,62.0,161.3,2019,4,hero_rb,1
62,Baker Mayfield,QB,65.0,229.2,2019,4,hero_rb,1
70,Alshon Jeffery,WR,74.0,122.2,2019,4,hero_rb,1
98,Kareem Hunt,RB,104.0,101.4,2019,4,hero_rb,1
102,Marvin Jones,WR,108.0,193.9,2019,4,hero_rb,1
118,Justin Jackson,RB,127.0,31.2,2019,4,hero_rb,1


In [14]:
top = top7(df)

In [15]:
top[top.draft_pos==4]

,player,pos,adp,ppr_pts,year,draft_pos,draft_strat,draft_num
62,Baker Mayfield,QB,65.0,229.2,2019,4,hero_rb,1
3,Ezekiel Elliott,RB,4.0,311.7,2019,4,hero_rb,1
127,Peyton Barber,RB,137.0,116.5,2019,4,hero_rb,1
45,Kenny Golladay,WR,48.0,248.0,2019,4,hero_rb,1
32,Amari Cooper,WR,35.0,246.5,2019,4,hero_rb,1
102,Marvin Jones,WR,108.0,193.9,2019,4,hero_rb,1
25,George Kittle,TE,28.0,222.5,2019,4,hero_rb,1


In [16]:
top.groupby(['draft_pos','draft_strat'])['ppr_pts'].sum()

draft_pos  draft_strat
1          hero_rb        1800.6
2          hero_rb        1346.6
3          hero_rb        1853.6
4          hero_rb        1568.3
5          zero_rb        1438.4
6          zero_rb        1498.7
7          zero_rb        1312.1
8          zero_rb        1519.2
9          value          1635.2
10         value          1570.0
11         value          1562.4
12         value          1358.2
Name: ppr_pts, dtype: object

In [17]:
top.shape

(82, 8)